In [1]:
# 1. Adım: Gerekli kütüphaneleri yükleme ve içe aktarma
!pip install --upgrade transformers accelerate -q

import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# -----------------------------------------------------
# Veri setinizi yükleyin
df = pd.read_csv('/kaggle/input/products-6000/balanced_shuffled_products.csv')
# -----------------------------------------------------

print("Kütüphaneler yüklendi.")

from huggingface_hub import notebook_login

print("Lütfen aşağıdaki kutucuğa Hugging Face token'ınızı girerek giriş yapın:")
notebook_login()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 75.8 MB/s eta 0:00:00:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.9 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.5 MB/s eta 0:00:00:00:0100:01
Kütüphaneler yüklendi.
Lütfen aşağıdaki kutucuğa Hugging Face token'ınızı girerek giriş yapın:


In [4]:
# 2. Adım: Tokenizer ve Modeli Manuel Olarak Yükleme (Önerilen Yöntemle)
model_name = "google/gemma-2b"
print(f"Tokenizer ve Model ({model_name}) indiriliyor...")
print("NOT: Bu işlem biraz zaman alabilir.")

# Başlangıçta None olarak ayarlıyoruz
tokenizer = None
model = None

try:
    # Tokenizer'ı yükle
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Modeli yükle: device_map="auto" ile modeli otomatik olarak uygun cihaza yerleştir.
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto", 
        torch_dtype=torch.bfloat16
    )
    
    print("Tokenizer ve Model başarıyla yüklendi!")
except Exception as e:
    print(f"Yükleme sırasında bir hata oluştu: {e}")
    print("\nLÜTFEN DİKKAT: Kaggle Secrets'a Hugging Face token'ınızı eklediğinizden ve internetin açık olduğundan emin olun.")

Tokenizer ve Model (google/gemma-2b) indiriliyor...
NOT: Bu işlem biraz zaman alabilir.


tokenizer_config.json:   0%|          | 0.00/33.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Tokenizer ve Model başarıyla yüklendi!


In [16]:
# 3. Adım: YALNIZCA model başarıyla yüklendiyse devam et
if tokenizer and model:
    # Test için örnekler
    sample_titles = df['title'][5:15].tolist()
    print("\n--- Açıklama Üretimi (One-Shot) Başladı ---\n")

    for i, title in enumerate(sample_titles):
        prompt = f"""
**Role:** You are an expert e-commerce copywriter. Your goal is to write a captivating and persuasive product description based on the example provided.

**Task:** Write a product description for the given product title, following the style and format of the examples.

**Example:**
**Product Title:** "AeroFlow Pro 5-in-1 Ionic Hair Dryer with Diffuser Attachment"
**Product Description:** "Achieve a flawless, salon-quality blowout at home with the AeroFlow Pro, which uses advanced ionic technology to reduce frizz and boost shine in record time."

**Product Title:** "{title}"
**Product Description:**
"""

        inputs = tokenizer(prompt, return_tensors="pt")

        outputs = model.generate(
            **inputs, 
            max_new_tokens=140,
            eos_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.3,
            top_p=0.95
        )
        
        full_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        
        try:
            generated_description = full_text.split("**Product Description:**")[-1].strip()
        except IndexError:
            generated_description = full_text[len(prompt):].strip()

        if tokenizer.eos_token:
             generated_description = generated_description.split(tokenizer.eos_token)[0].strip()

        print(f"--- Örnek {i+1} ---")
        print(f"Orijinal Başlık: {title}")
        print(f"-> Üretilen Açıklama: {generated_description}\n")
else:
    print("\nModel veya Tokenizer yüklenemediği için açıklama üretme adımı atlandı.")


--- Açıklama Üretimi (One-Shot) Başladı ---

--- Örnek 1 ---
Orijinal Başlık: StrikerDC Dreamcast Controller - Gray
-> Üretilen Açıklama: "Are you looking to take your digital marketing skills to the next level? Look no further than The Ultimate Guide to Digital Marketing! This comprehensive

--- Örnek 2 ---
Orijinal Başlık: Power Supply for PS2, Replacement AC Adapter Charger Cord for Sony Playstation 2 PS2 Slim A/C 70000 Gaming Console
-> Üretilen Açıklama: "The Power Supply for PS2 is a replacement AC adapter charger cord for Sony Playstation 2 PS2 Slim A/C 70000 Gaming Console. This high-quality power supply features a built-in surge protector to protect your console from power surges and other electrical disturbances. It also has a built-in safety fuse to prevent overheating and short-circuiting. The Power Supply for PS2 is compatible with all models of the Sony Playstation 2 PS2 Slim A/C 70000 Gaming Console, and it comes with a 1-year warranty for added peace of mind. Order now